In [1]:
%%capture
%pip install llama-index-embeddings-huggingface
%pip install llama-index-llms-huggingface
%pip install llama-index

In [3]:
from llama_index.core import Settings, SQLDatabase
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)


In [4]:
engine = create_engine("sqlite:///:memory:", future=True)
metadata_obj = MetaData()

In [5]:
# create city SQL table
table_name = "medicine"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("med_id", Integer , primary_key=True),
    Column("med_name", String(16)),
    Column("price", Integer),
    Column("uses", String(30), nullable=False),
    Column("side_effects", String(30)),
)

metadata_obj.create_all(engine)

In [6]:
from sqlalchemy import insert

rows = [
    {"med_id": 1, "med_name": "paracetamol", "price": 29, "uses": "Pain relief", "side_effects": "None"},
    {"med_id": 2, "med_name": "aspirin", "price": 13, "uses": "Pain relief, Fever reduction", "side_effects": "Stomach irritation"},
    {"med_id": 3, "med_name": "ibuprofen", "price": 36, "uses": "Pain relief, Inflammation reduction", "side_effects": "Stomach irritation, Headache"},
    {"med_id": 4, "med_name": "acetaminophen", "price": 25, "uses": "Pain relief, Fever reduction", "side_effects": "Liver damage"},
    {"med_id": 5, "med_name": "naproxen", "price": 20, "uses": "Pain relief, Inflammation reduction", "side_effects": "Stomach irritation, Heart attack risk"},
    {"med_id": 6, "med_name": "diphenhydramine", "price": 15, "uses": "Allergy relief, Insomnia treatment", "side_effects": "Drowsiness, Dry mouth"},
    {"med_id": 7, "med_name": "loratadine", "price": 18, "uses": "Allergy relief", "side_effects": "Headache, Dry mouth"},
    {"med_id": 8, "med_name": "ranitidine", "price": 22, "uses": "Acid reflux treatment", "side_effects": "Headache, Constipation"},
    {"med_id": 9, "med_name": "omeprazole", "price": 30, "uses": "Acid reflux treatment", "side_effects": "Headache, Nausea"},
    {"med_id": 10, "med_name": "simvastatin", "price": 40, "uses": "Cholesterol management", "side_effects": "Muscle pain, Liver damage"}
]

for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [7]:
with engine.connect() as connection:
    cursor = connection.exec_driver_sql("SELECT * FROM medicine")
    print(cursor.fetchall())

[(1, 'paracetamol', 29, 'Pain relief', 'None'), (2, 'aspirin', 13, 'Pain relief, Fever reduction', 'Stomach irritation'), (3, 'ibuprofen', 36, 'Pain relief, Inflammation reduction', 'Stomach irritation, Headache'), (4, 'acetaminophen', 25, 'Pain relief, Fever reduction', 'Liver damage'), (5, 'naproxen', 20, 'Pain relief, Inflammation reduction', 'Stomach irritation, Heart attack risk'), (6, 'diphenhydramine', 15, 'Allergy relief, Insomnia treatment', 'Drowsiness, Dry mouth'), (7, 'loratadine', 18, 'Allergy relief', 'Headache, Dry mouth'), (8, 'ranitidine', 22, 'Acid reflux treatment', 'Headache, Constipation'), (9, 'omeprazole', 30, 'Acid reflux treatment', 'Headache, Nausea'), (10, 'simvastatin', 40, 'Cholesterol management', 'Muscle pain, Liver damage')]


In [8]:
#Download the ebedding model for the llm
Settings.embed_model = HuggingFaceEmbedding( model_name= "sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
sql_database = SQLDatabase(engine, include_tables=["medicine"])

In [26]:
#initialise llm
from llama_index.llms.huggingface import HuggingFaceLLM



llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    # model_kwargs={"torch_dtype": torch.float16, }
)

In [27]:
#create query engine for text to sql
from llama_index.core.query_engine import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["medicine"], llm=llm
)


In [29]:
#Quering to DataBase
response = sql_query_engine.query(
    "what is the uses of diphenhydramine?"
)

print(str(response))

Diphenhydramine is a medication that is used to treat allergies, insomnia, and other conditions. The medication works by reducing the production of histamine in the body, which can help relieve allergy symptoms, sleep disturbances, and other conditions.
